In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import operator

In [ ]:
content = open("missionspage.html","rb").read()
b= BeautifulSoup(content,"html5lib")

In [ ]:
allMissionSets = {}

for s in b.html.body.find_all(id="missions-overview")[0].find_all("div","set-container"):    
    srow   = s.find_all("div","set-info")[0]
    sid    = int(s["data-id"])
    stitle = "".join(c.strip() for c in srow.contents if isinstance(c,str))    
    if stitle.endswith(" Missions"):
        stitle = stitle[:-9]
    
    missions = {}
    for m in s.find_all("div","mission-container"):
        mid    = int(m["data-id"])
        mrow   = m.find_all("div","mission-row")[0]
        mtitle = "".join(c.strip() for c in mrow.contents if isinstance(c,str))    
        
        mtasks = []        
        if "locked" not in m["class"]:
            for t in m.find_all("div","objectives")[0].find_all("li"):
                mtasks.append("".join(c.strip() for c in t.contents if isinstance(c,str)))        
        
        missions[mid] = (mtitle,mtasks)
    
    allMissionSets[sid] = (stitle, missions)

In [ ]:
activeMissions = [] 
for m in b.html.body.find_all(id="missions-active")[0].find_all("div","mission-container"):    
    id    = int(m["data-id"])
    activeMissions.append(id)

In [ ]:
ALLAREAS = {
    "Hemmeldal"        : ("Moose","Brown bear","Reindeer","Roe deer","Red fox","Eurasian Lynx",
                          "Ptarmigan", "Willow ptarmigan"),
    "Whitehart"        : ("Roosevelt elk","Whitetail","Blacktail","Coyote","Bobcat","Turkey"),
    "Logger's Point"   : ("Feral hog","Mule","Whitetail","Coyote","Bobcat","Pheasant","Cottontail rabbit"),
    "Settler Creeks"   : ("Roosevelt elk","Black bear","Feral hog","Whitetail","Bobcat","Turkey","Cottontail rabbit"),
    "Redfeather Falls" : ("Moose","Black bear","Roosevelt elk","Whitetail","Blacktail"),
    "Hirschfelden"     : ("Red deer","Wild boar","Roe deer","Red fox","Canada goose","Pheasant"),
    "Rougarou Bayou"   : ("Black bear","Feral hog","Whitetail","Bobcat",
                          "Mallard","American black duck","Northern pintail","Gadwall"),
    "Val-des-Bois"     : ("Black bear","Red deer","Alpine ibex","Roe deer","Red fox","European rabbit",
                          "Ptarmigan", "Rock ptarmigan"),
    "Bushrangers Run"  : ("Feral hog","Red kangaroo","Feral goat","Red fox","European rabbit"),
    "Whiterime Ridge"  : ("Bison","Moose","Polar bear","Sitka deer","Dall sheep","Arctic fox","Snowshoe hare",
                          "Snow goose"),
    "Timbergold Trails": ("Rocky mountain elk","Brown bear","Mule deer","Grey wolf","Puma","Bighorn sheep",
                          "Ptarmigan", "White-tailed ptarmigan"),
    "Piccabeen Bay"    : ("Water buffalo","Banteng","Sambar deer","Rusa deer","Feral hog","Magpie goose"),
}

ADDITIONAL = ("Shotgun",
              "Laura Francese",
              #"Blaser",
              "Anschütz",
              #"Tenpoint",
              #"Parker Bows",
              #"Nosler",
              "The Boone And Crockett Club",
              #"Aimpoint",
             )
NEW_WEAPONS = (".300 Nosler AccuBond",
               "8x57 IS Anschütz 1780 D FL Bolt Action Rifle",
               "Parker Python Compound Bow",
               ".223 Bolt Action Rifle",
               ".300 Rifle",
               "12 GA Blaser F3 Game O/U Shotgun",
               "Tenpoint Carbon Fusion Crossbow",
               ".22 Air Rifle",
               "Aimpoint sight",
               "Longbow",
              )

In [ ]:
def checkKeywords(kws,titles):
    for kw in kws:
        kw = kw.lower()
        for ti in titles:
            ti = ti.lower()
            if kw in ti:
                return True
    return False

In [ ]:
areas        = set(("Hirschfelden",))
#areas        = set(ALLAREAS.keys())

animals      = set().union(*map(lambda x: ALLAREAS[x],areas))

allanimals   = set().union(*ALLAREAS.values())
otheranimals = allanimals - animals
    
allareas     = set(ALLAREAS.keys())
otherareas   = allareas - areas
    
goodKeywords = areas      | animals      | set(ADDITIONAL) 
badKeywords  = otherareas | otheranimals 
markKeywords = set(NEW_WEAPONS)

mindex = []
sindex = []
iindex = []
data   = []
for m in activeMissions:    
    for s,(st,ml) in allMissionSets.items():
        for sm,(mt,mk) in ml.items():
            if sm == m:
                sid      = s
                stitle   = st
                missions = ml                            
    mtitle = missions[m][0]        
    mtasks = missions[m][1]
   
    if not checkKeywords(goodKeywords,(stitle,)+tuple(mtasks)):
        continue
        
    if     checkKeywords(badKeywords, (stitle,)+tuple(mtasks)):
        continue

    bad = False
    if     checkKeywords(markKeywords, (stitle,)+tuple(mtasks)):
        bad = True       
                
    for i,tk in enumerate(mtasks):
        mindex.append(mtitle)
        sindex.append(stitle)
        iindex.append(i)
        data.append([tk,not bad])
                
idx = pd.MultiIndex.from_arrays((mindex,sindex,iindex))
df = pd.DataFrame(data,index=idx,columns=["Task","Ready"])
df = df.sort_values("Ready")
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',     50)
df